**Part 3 - Mapping Closure in 2D for f_ZB**

This notebook implements the mapping closure developed by (Chen, H. 1989) and extended by (Pope, S.B. 1991) for a two-dimensional (2D) problem where the vector of random variables $\mathbf{Y} = (Z,B)$ inlcues the vertical height and the buoyancy $B$.


*Evolution equation*

The joint pdf $f_{\mathbf{Y}}$ evolves according to the forward Kolmogorov equation given by

\begin{equation}
\frac{\partial f_{\mathbf{Y}} }{ \partial t}= -\frac{\partial }{\partial y_i} \mathbf{D}^{(1)}_i f_{\mathbf{Y}}  + \frac{\partial^{2}}{\partial y_i \partial y_j} \mathbf{D}^{(2)}_{ij} f_{\mathbf{Y}},
\end{equation}

where 

\begin{equation}
  \mathbf{D}^{(1)}=
  \phi\,\mathbb{E}_{\mathbf{Y}_{t}|\partial\Omega}
 \begin{bmatrix}
 0 \\
 \alpha_{1}\,\mathbf{n}\cdot\nabla B_{t} \\
 \end{bmatrix}
  \frac{f_{\mathbf{Y}|\partial\Omega}}{f_{\mathbf{Y}}} .
\end{equation}

arises due to diffusive fluxes at the boundary, which must be specified. The diffusion term for our system us given by either
\begin{equation}
  \mathbf{D}^{(2)}=-\alpha_{1}
  \mathbb{E}_{\mathbf{Y}_{t}}
    \begin{bmatrix}
    -1 & 0 \\
    0 & |\nabla B_{t}|^{2}\\
  \end{bmatrix},
\end{equation}
or
\begin{equation}
  \mathbf{D}^{(2)}=-\alpha_{1}
  \mathbb{E}_{\mathbf{Y}_{t}}
    \begin{bmatrix}
    0 & \partial_Z B_t/2 \\
    \partial_Z B_t/2  & |\nabla B_{t}|^{2}\\
  \end{bmatrix}.
\end{equation}
However to avail of the mapping closure (Pope, S.B. 1991) we will use to model these terms we will make the assumption that
\begin{equation}
  \frac{\partial^{2}}{\partial y_i \partial y_j} \mathbf{D}_{ij}^{(2)} f_{\mathbf{Y}} \approx -\frac{\partial }{ \partial y_i} \mathbb{E}_{\mathbf{Y}_{t}} [ \alpha_{ij} \nabla^{2} \mathbf{Y}^j_{t}] f_{\mathbf{Y}},
\end{equation}
which at least for the 1D problem has been shown to be equivalent. For now we consider only the unforced system when $\mathbf{D}^{(1)}_{i} = 0$ before later reintroducing these terms.

*Mapping closure*

Corresponding to the random vector $\mathbf{Y} = (Y_1,Y_2)$ we now have a vector of Gaussian random fields $\mathbf{Z} = (Z_1,Z_2)$. The (now multidimensional) mapping $\mathscr{Y}_{\alpha}(-,t):\mathbb{R}^{\alpha} \rightarrow \mathbb{R}$ which maps the Gaussian field(s) to the surrogate field $\tilde{Y}_{\alpha}$ is hierarchical in the sense that 
\begin{align*}
    \tilde{Y}_1(\mathbf{x},t) &= \mathscr{Y}_{1}(Z_1(J_1(t)\mathbf{x}),t), \\
    \tilde{Y}_2(\mathbf{x},t) &= \mathscr{Y}_{2}(Z_1(J_1(t)\mathbf{x}),Z_2(J_2(t)\mathbf{x}),t),
\end{align*}
such that $\mathscr{Y}_{\alpha}$ only depends on the subset $[\mathbf{Z}]_{\alpha}$

*Mapping equation*

The mapping $\mathscr{Y}_{\alpha}(-,t)$ evolves according to

\begin{equation}
\frac{\partial \mathscr{Y}_{\alpha} }{\partial t} = \mathbf{D}^{(1)}_{\alpha} + \sum_{j=1}^{\alpha} \frac{\gamma_{\alpha}}{\tau_{j}(t)} \left( \frac{\partial^{2}}{\partial z_{j}^{2}} - z_{j}\frac{\partial }{ \partial z_{j} }\right)\mathscr{Y}_{\alpha},
\end{equation}

however for a simple domain such as a rectangular box we know that $f_Z(z;t)$ is independent of time and is given by the uniform distribution $f_Z(z;t) = 1/L_z$. This implies that $\mathscr{Y}_1(z_1,t) = G(z_1)$, where $G$ denotes the conditional cumulative normal distribution and that we do not need to model this equation. 

When expanded the previous equation is therefore given by

\begin{equation}
\frac{\partial \mathscr{Y}_2 }{\partial t} = \gamma_2 \left[ \frac{1}{\tau_1(t)} \left( \frac{\partial^{2}}{\partial z_{1}^{2}} - z_1\frac{\partial }{ \partial z_1 }\right) + \frac{1}{\tau_2(t)} \left( \frac{\partial^{2}}{\partial z_{2}^{2}} - z_2\frac{\partial }{ \partial z_2 }\right) \right] \mathscr{Y}_2,
\end{equation}
with $\tau_1(t) = 1$ as there is no stretching of this variable.
